In [77]:
import math

class TreeNode:
    def __init__(self, attribute=None, label=None):
        self.attribute = attribute  # Attribute to split on
        self.label = label  # Class label if leaf node, None otherwise
        self.children = {}  # Dictionary of attribute values to child nodes

<b>Finding out the Entropy</b> (a measure of disorder or impurity in a node. ) Of the DT.
![Alt text](image-2.png)

In [78]:
def entropy(data):
    class_counts = {}# Dictionary of class labels to counts
    #In this loop we are counting the number of each class label
    for entry in data:
        label = entry[-1]
        if label not in class_counts:
            class_counts[label] = 0#If the label is not in the dictionary, we add it
        class_counts[label] += 1#increment the count of the label

    total_entries = len(data)
    entropy_value = 0 #the entropy value is initialised to zero.

    for count in class_counts.values():
        probability = count / total_entries
        entropy_value -= probability * math.log2(probability)

    return entropy_value


![Alt text](image-4.png)
# Next,calculating the I.G. of each feature and selecting the one with the highest I.G. value.
![Alt text](image-3.png)

In [79]:
#This function calculates the information gain for a given attribute.
def information_gain(data, attribute_index):
    attribute_values = {}#dictionary of attribute values to data subsets
    #In this loop we are splitting the data into subsets based on the attribute values
    for entry in data:
        value = entry[attribute_index]
        if value not in attribute_values:
            attribute_values[value] = []
        attribute_values[value].append(entry)

    total_entries = len(data)
    attribute_entropy = 0

    for value_data in attribute_values.values():
        proportion = len(value_data) / total_entries
        attribute_entropy += proportion * entropy(value_data)

    return entropy(data) - attribute_entropy

# Function used for Building and Printing the D.T.

In [80]:

def build_decision_tree(data, attributes):
    class_labels = set(entry[-1] for entry in data)#set of unique class labels

    # If all instances have the same class label
    if len(class_labels) == 1:
        return TreeNode(label=class_labels.pop())

    # If no attributes left or data is pure and If true, create a leaf node with the majority class label and return it.
    if not attributes or entropy(data) == 0:
        label_counts = {label: 0 for label in class_labels}#dictionary of class labels to counts
        for entry in data:
            label_counts[entry[-1]] += 1
        majority_label = max(label_counts, key=label_counts.get)
        return TreeNode(label=majority_label)

    # Find the attribute with the highest information gain
    best_attribute_index = max(range(len(attributes)), key=lambda i: information_gain(data, i))
    best_attribute = attributes[best_attribute_index]
    # Create a new decision tree node with the best attribute
    root = TreeNode(attribute=best_attribute)
    attribute_values = set(entry[best_attribute_index] for entry in data)
    # Create a new decision tree sub-node for each of the values in the best attribute field.
    for value in attribute_values:
        subset_data = [entry for entry in data if entry[best_attribute_index] == value]
        if not subset_data:
            label_counts = {label: 0 for label in class_labels}
            for entry in data:
                label_counts[entry[-1]] += 1
            majority_label = max(label_counts, key=label_counts.get)
            root.children[value] = TreeNode(label=majority_label)
        else:
            remaining_attributes = [attr for i, attr in enumerate(attributes) if i != best_attribute_index]
            root.children[value] = build_decision_tree(subset_data, remaining_attributes)

    return root

def print_tree(node, indent=""):
    if node.label is not None:
        print(indent + "Class:", node.label)
    else:
        print(indent + "Attribute:", node.attribute)
        for value, child_node in node.children.items():
            print(indent + "Value:", value)
            print_tree(child_node, indent + "  ")



In [81]:
def main():
    # Example dataset: outlook, temperature, humidity, windy, play
    data = [
        ('sunny', 'hot', 'high', False, 'no'),
        ('sunny', 'hot', 'high', True, 'no'),
        ('overcast', 'hot', 'high', False, 'yes'),
        ('rainy', 'mild', 'high', False, 'yes'),
        ('rainy', 'cool', 'normal', False, 'yes'),
        ('rainy', 'cool', 'normal', True, 'no'),
        ('overcast', 'cool', 'normal', True, 'yes'),
        ('sunny', 'mild', 'high', False, 'no'),
        ('sunny', 'cool', 'normal', False, 'yes'),
        ('rainy', 'mild', 'normal', False, 'yes'),
        ('sunny', 'mild', 'normal', True, 'yes'),
        ('overcast', 'mild', 'high', True, 'yes'),
        ('overcast', 'hot', 'normal', False, 'yes'),
        ('rainy', 'mild', 'high', True, 'no')
    ]

    attributes = ['outlook', 'temperature', 'humidity', 'windy']

    decision_tree = build_decision_tree(data, attributes)
    print_tree(decision_tree)

if __name__ == "__main__":
    main()

Attribute: outlook
Value: overcast
  Class: yes
Value: rainy
  Attribute: humidity
  Value: cool
    Attribute: temperature
    Value: rainy
      Attribute: windy
      Value: rainy
        Class: no
  Value: mild
    Attribute: temperature
    Value: rainy
      Attribute: windy
      Value: rainy
        Class: yes
Value: sunny
  Attribute: windy
  Value: normal
    Class: yes
  Value: high
    Class: no


# So final Interpretation-

<ol>
<li>The root node splits the data based on the "outlook" attribute.</li>
<li>If the outlook is "overcast," the class is predicted as "yes."</li>
<li>If the outlook is "rainy," further splitting is done based on "humidity" and "temperature."</li>
<li>If the humidity is "cool" and the temperature is "rainy," the decision is based on the "windy" attribute.</li>
<li>The process continues recursively until leaf nodes are reached, representing the predicted class.</li>
</ol>